In [2]:
import bencode
import hashlib

import requests
from urllib.parse import urlencode

import struct
import socket

import random
import string 

In [3]:
def getPeerID(N = 20) :
    peerId = "-"
    peerId += "MU0707-"
    peerId += ''.join(random.choices(string.ascii_letters + string.digits, k = (N - 8)))
    #print(peerId)
    return peerId

In [4]:
def decodeTorrent(filename) :
    #do you want port number as user input?
    if filename.split('.')[-1] != "torrent" :
        raise Exception("Please enter .torrent file.")
        return tuple()
    else :
        torrentFile = open(filename, "rb")
        metainfo = bencode.bdecode(torrentFile.read())
        try :
            tracker_url = metainfo['announce']
            tracker_url_list = metainfo['announce-list']
            info = metainfo['info']
            info_hash = hashlib.sha1(bencode.encode(info)).digest()
            if 'files' in info :
                isFolder = True
            else :
                isFolder = False
        except :
            raise Exception("File not bencoded properly.")
    return (tracker_url, tracker_url_list, info_hash, info, isFolder)

In [22]:
def HTTPTrackerRequest(tracker_url, tracker_url_list, info_hash, info, isFolder) :
    if not isFolder :
        name = info['name']
        length = info['length']
    else :
        files = info['files']
        name = info['name']
        
        #loop through files......every time one get request??
        length = files[1]['length']
        path = files[1]['path']
    port = 6881
    peer_id = '-MU0707-wASPVhj5UpMd'
    #peer_id = getPeerID(20)
    query_params = urlencode({"info_hash" : info_hash, "peer_id" : peer_id, "port" : port, \
                              "uploaded" : 0, "downloaded" : 0, "left" : length, \
                              "event" : "started",\
                              "numwant" : 50
                             })
    lookup_url = f"{tracker_url}?{query_params}"
    r = requests.get(lookup_url)
    content = r.content
    if content == b'<title>Invalid Request</title>\n' :
        return {"Error " : "Invalid Rquest"}
    #print(content)
    return dict(bencode.bdecode(content))

In [8]:
def getPeers(response) :
    #response as a dictionary
    if 'Error' in response or 'failure reason' in response :
        return [] 
    returnList = []
    if type(response['peers']) == list :
        returnList = response['peers']
    else :
        try :
            binary_ip = response['peers'] 
            offset = 0
            while offset != n :
                ip1 = struct.unpack_from("!i", binary_ip, offset)[0] # ! = network order(big endian); i = int
                ip = socket.inet_ntoa(struct.pack("!i", ip1))
                offset += 4 # save where the first ip ends and the port begins
                port = struct.unpack_from("!H", binary_ip, offset)[0] # H = unsigned short
                returnList.append({"ip" : ip, "port" : port})
                offset += 2
        except :
            pass
    return returnList

In [25]:
x, y, z, a, b = decodeTorrent("ubuntu-20.04.1-desktop-amd64.iso.torrent")

In [26]:
list1 = getPeers(HTTPTrackerRequest(x, y, z, a, b))

In [27]:
list1

[OrderedDict([('ip', '217.211.190.116'),
              ('peer id', '-DE203s-au30R6xjCMtb'),
              ('port', 61111)]),
 OrderedDict([('ip', '82.64.200.167'),
              ('peer id', '-qB4030-7QNcXbZVLQzc'),
              ('port', 51413)]),
 OrderedDict([('ip', '71.150.198.66'),
              ('peer id', '-TR2940-c1yx5z3d5ach'),
              ('port', 51413)]),
 OrderedDict([('ip', '76.201.6.148'),
              ('peer id', '-TR2940-a0h00o2feq00'),
              ('port', 51413)]),
 OrderedDict([('ip', '62.210.88.76'),
              ('peer id', b'-lt0D60-\x87f\x12 ;\xafd\x8e\xe7`o\xad'),
              ('port', 6897)]),
 OrderedDict([('ip', '185.148.3.88'),
              ('peer id', b'-lt0D80-\x08\xb3\xd2Y\x94\xe4\x8a.\xb0\xc1@/'),
              ('port', 57396)]),
 OrderedDict([('ip', '91.189.95.21'),
              ('peer id', 'T03I--00sAYN4OiaMX7O'),
              ('port', 6902)]),
 OrderedDict([('ip', '79.1.33.180'),
              ('peer id', '-TR2940-jre86u1ezgqk'),
           

In [28]:
len(list1)

50